In [21]:
from method import *
from public import *


In [42]:
import math
df_单位净值=pd.read_csv(Public.readFile("mid_1.csv"), index_col=0)
lst_最新一天单位净值 = []
lst_最新一天的日期 = []
for j in range(2,df_单位净值.shape[1]):
    idx = np.where(df_单位净值.iloc[:,j] !=0.0)[0][-1]
    lst_最新一天单位净值 = lst_最新一天单位净值 + [df_单位净值.iloc[idx,j]]
    lst_最新一天的日期 = lst_最新一天的日期 + [df_单位净值.index[idx]]
    
df_最新一天单位净值 = pd.DataFrame(df_单位净值.iloc[-1,2:].T)  #先粗略地用最后一行的代表最新的单位净值
df_最新一天单位净值["产品代码"] = df_最新一天单位净值.index
df_最新一天单位净值.columns = ["最新一天单位净值","产品代码"]
df_最新一天单位净值["最新一天单位净值"] = lst_最新一天单位净值  #再替换成真正的最新一天的单位净值
df_最新一天单位净值["最新一天的日期"] = lst_最新一天的日期
df_最新一天单位净值["最新一天的日期"] = [str(i).split(" ")[0] for i in df_最新一天单位净值["最新一天的日期"]]


,最新一天单位净值,产品代码,最新一天的日期
中证800,4312.3442,中证800,747
SGD165,1.224,SGD165,747
SX1811,1.6,SX1811,747
SGW516,1.9259,SGW516,747
GL921B,1.2868,GL921B,747
...,...,...,...
SXM115,0,SXM115,747
SXP967,0,SXP967,747
SXR443,0.9989,SXR443,747
SLW925,0,SLW925,747


In [43]:
df_最新一天单位净值

,最新一天单位净值,产品代码,最新一天的日期
中证800,4312.3442,中证800,747
SGD165,1.224,SGD165,747
SX1811,1.6,SX1811,747
SGW516,1.9259,SGW516,747
GL921B,1.2868,GL921B,747
...,...,...,...
SXM115,0,SXM115,747
SXP967,0,SXP967,747
SXR443,0.9989,SXR443,747
SLW925,0,SLW925,747


In [44]:
#开始填充底层资产sample
df_底层资产sample = pd.read_excel(Public.readFile("持仓日报表模板.xlsx"),sheet_name="底层资产模板")
df_底层资产sample.index = df_底层资产sample["产品代码"]
for i in range(0,df_底层资产sample.shape[0]):
    产品代码 = df_底层资产sample.iloc[i,1]
    df_底层资产sample.iloc[i,10] = float(df_最新一天单位净值.loc[df_最新一天单位净值["产品代码"] == 产品代码]["最新一天单位净值"]) 
#double
df_底层资产sample["成本价（元/份）"] = df_底层资产sample["投资金额（万元）"]/df_底层资产sample["持有份额（份）"]*10000
df_底层资产sample["持仓市值（万元）"] = df_底层资产sample["持有份额（份）"] * df_底层资产sample["单位净值（元/份）"]/10000

持仓总市值 = df_底层资产sample["持仓市值（万元）"].sum()
df_底层资产sample["持仓市值占总持仓比例"] = df_底层资产sample["持仓市值（万元）"] /持仓总市值
f1 = lambda x :'%.2f%%'  %  (x*100)
df_底层资产sample[["持仓市值占总持仓比例"]] = df_底层资产sample[["持仓市值占总持仓比例"]].applymap(f1)
df_底层资产sample["2022年浮动盈亏（万元）"]= (df_底层资产sample["单位净值（元/份）"] - df_底层资产sample["2021年12月31日单位净值"])*df_底层资产sample["持有份额（份）"]/10000

df_底层资产sample["已实现盈亏（万元）"].fillna(0,inplace=True)
df_底层资产sample["2022年盈亏比例"] = (df_底层资产sample["2022年浮动盈亏（万元）"]+df_底层资产sample["已实现盈亏（万元）"])/df_底层资产sample["投资金额（万元）"]
df_底层资产sample[["2022年盈亏比例"]] = df_底层资产sample[["2022年盈亏比例"]].applymap(f1)
df_底层资产sample["累计浮动盈亏（万元）"] = (df_底层资产sample["单位净值（元/份）"] - df_底层资产sample["成本价（元/份）"])*df_底层资产sample["持有份额（份）"]/10000

df_底层资产sample["累计盈亏比例"] = (df_底层资产sample["累计浮动盈亏（万元）"]+df_底层资产sample["已实现盈亏（万元）"])/df_底层资产sample["投资金额（万元）"]
df_底层资产sample[["累计盈亏比例"]] = df_底层资产sample[["累计盈亏比例"]].applymap(f1)


series_各管理人持仓 = df_底层资产sample.groupby(by=["产品管理人"])["投资金额（万元）"].sum()
df_各管理人持仓 = series_各管理人持仓.reset_index()
for i in range(0,df_底层资产sample.shape[0]):
    产品管理人 = df_底层资产sample.iloc[i,3]
    df_底层资产sample.iat[i,-5]= df_各管理人持仓.loc[df_各管理人持仓["产品管理人"] == 产品管理人]["投资金额（万元）"]

for i in range(0,df_底层资产sample.shape[0]):
    if math.isnan(df_底层资产sample["产品规模（总份额）"].iloc[i]):
        df_底层资产sample.iloc[i,-3]=df_底层资产sample["产品规模（总份额）"].iloc[i]
    else:
        df_底层资产sample.iloc[i,-3] = format((df_底层资产sample["持有份额（份）"].iloc[i] / df_底层资产sample["产品规模（总份额）"].iloc[i]),".2%")
        

In [45]:
#填充号的底层资产sample就是底层资产私募配置情况表了
df_底层资产私募配置情况 = df_底层资产sample
df_hh = df_底层资产私募配置情况.fillna(0,inplace=False)
idx = []
for i in range(0,64):
    if df_hh.iloc[i,0] != 0:
        idx = idx + [i]
    else:
        df_hh.iloc[i,0] = df_hh.iloc[i,0]
        
for i in range(0,idx[1]):
    df_hh.iloc[i,0] = "一对一产品"
for i in range(idx[1],df_hh.shape[0]):
    df_hh.iloc[i,0] = "一对多产品"

df_底层资产私募配置情况.iloc[:,0] = df_hh.iloc[:,0]
# df_底层资产私募配置情况
#df_底层资产私募配置情况.head(2)

,私募基金产品类型,产品代码,产品名称,产品管理人,策略类型,细分策略,投资金额（万元）,持有份额（份）,成本价（元/份）,2021年12月31日单位净值,...,累计浮动盈亏（万元）,累计盈亏比例,产品到期日,已实现盈亏（万元）,赎回金额（万元）,管理人持仓,产品规模（总份额）,持有份额 /产品规模（总份额）,营业部,IC
产品代码,,,,,,,,,,,,,,,,,,,,,
SQZ005,一对一产品,SQZ005,凡二量化对冲9号私募证券投资基金,深圳凡二投资管理合伙企业（有限合伙）,量化对冲,中性,4400.00000,42765262.55,1.028872,1.0585,...,-56.332283,-1.28%,2023-08-13 00:01:00,0.0,NaN,4400.00000,NaN,NaN,北京建国门外大街,张潇丹
SSM689,一对一产品,SSM689,星阔金选云起12号市场中性,星阔（青岛）投资管理有限公司,量化对冲,中性,3500.00000,35045559.23,0.998700,0.9720,...,-127.215380,-3.63%,2023-09-10 00:00:00,0.0,NaN,4000.00000,NaN,NaN,深圳分公司,万小妹
STG510,一对一产品,STG510,橡木成林三号,杭州橡木资产管理有限公司,量化对冲,中性,3000.00000,30030030.03,0.999000,1.0060,...,150.150150,5.01%,2022-12-15 00:00:00,0.0,NaN,3000.00000,NaN,NaN,深圳分公司,罗涛
STF617,一对一产品,STF617,衍合金选量化市场中性1号,衍合投资管理（青岛）有限公司,量化对冲,中性,3000.00000,29734255.55,1.008937,1.0050,...,-41.441573,-1.38%,2022-12-29 00:00:00,0.0,NaN,3000.00000,NaN,NaN,深圳嘉里建设广场,王新宇
STM651,一对一产品,STM651,百创金长牛1号,深圳前海百创资本管理有限公司,股票多头,主观多头,500.00000,5005005.01,0.999000,0.9990,...,-98.598598,-19.72%,2023-01-19 00:00:00,0.0,NaN,500.00000,NaN,NaN,广州珠江新城,李凡平
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GU147A,一对多产品,GU147A,中量投精合8号私募证券投资基金,中量投资产管理有限公司,多策略灵活配置,多策略（股票+CTA+期权+固收),500.00000,4629629.63,1.080000,1.0748,...,57.268519,11.45%,2023-06-30 00:00:00,0.0,NaN,500.00000,6.909138e+07,6.70%,深圳爱国路,仝链
SGX161,一对多产品,SGX161,赋源多策略量化五号私募证券投资基金,上海赋源投资管理有限公司,多策略灵活配置,多策略（股票+中性）,200.00000,1754232.08,1.140100,1.1854,...,-23.138322,-11.57%,2023-06-29 00:00:00,0.0,NaN,200.00000,1.371820e+07,12.79%,深圳分公司,张雯
SGB123,一对多产品,SGB123,蒙玺祖冲之1号量化私募证券投资基金,上海蒙玺投资管理有限公司,指数增强,量化选股,200.00000,1122964.63,1.781000,2.1860,...,34.811904,17.41%,2023-04-30 00:00:00,0.0,NaN,200.00000,1.921261e+07,5.84%,北京北三环东路,陈丹（周楠）


In [26]:
df_金富1号 = pd.read_excel(Public.readFile("持仓日报表模板.xlsx"),sheet_name="金富1号模板")
df_金富1号 = df_金富1号.iloc[[0,1],:]
df_金富1号.columns = df_金富1号.iloc[0]
df_金富1号.drop(df_金富1号.index[:1],inplace=True) 
df_金富1号

,FOF单一资管计划,委托财产,持有份额,2022年初资产净值+追加资产,单位净值,当前资产净值,盈亏
1,金富1号,92379.56,95599.868304,95927.851002,0.967,92430.418688,-3497.432314


In [27]:
#开始填充盈亏情况表、策略分布表

#创建空表格
df_业务资产及盈亏情况 = pd.read_excel(Public.readFile("持仓日报表模板.xlsx"), sheet_name="盈亏情况模板")
df_业务资产及盈亏情况.columns =["指标","数值","指标","数值"]
#计算的数据
df_业务资产及盈亏情况.iloc[2,1] = float(df_金富1号["委托财产"]) + sum(df_底层资产私募配置情况["投资金额（万元）"][-2:])
df_业务资产及盈亏情况.iloc[1,1] = df_业务资产及盈亏情况.iloc[0,1] - df_业务资产及盈亏情况.iloc[2,1]
df_业务资产及盈亏情况.iloc[3,1] = sum(df_底层资产私募配置情况["2022年浮动盈亏（万元）"][0:idx[1]])
df_业务资产及盈亏情况.iloc[4,1] = sum(df_底层资产私募配置情况["持仓市值（万元）"][0:idx[1]])
df_业务资产及盈亏情况.iloc[0,3] = format(df_业务资产及盈亏情况.iloc[1,1]/df_业务资产及盈亏情况.iloc[0,1],".2%")
#df_业务资产及盈亏情况.iloc[2,3] = sum(df_底层资产私募配置情况["2022年浮动盈亏（万元）"][idx[1]:])

df_业务资产及盈亏情况.iloc[2,3] = sum(df_底层资产私募配置情况["2022年浮动盈亏（万元）"].fillna(0)[idx[1]:])


df_业务资产及盈亏情况.iloc[3,3] = float(df_金富1号["盈亏"]) + df_业务资产及盈亏情况.iloc[-1,-1]+ sum(df_底层资产私募配置情况["2022年浮动盈亏（万元）"][-2:])
#df_业务资产及盈亏情况.iloc[4,3] = sum(df_底层资产私募配置情况["持仓市值（万元）"][idx[1]:])

df_业务资产及盈亏情况.iloc[4,3] = sum(df_底层资产私募配置情况["持仓市值（万元）"].fillna(0)[idx[1]:])

df_业务资产及盈亏情况.iloc[1,3] = format(df_业务资产及盈亏情况.iloc[3,3] / df_业务资产及盈亏情况.iloc[0,1],".2%")
df_业务资产及盈亏情况.iloc[5,1] = df_业务资产及盈亏情况.iloc[4,1] + df_业务资产及盈亏情况.iloc[4,3]

df_业务资产及盈亏情况 = df_业务资产及盈亏情况.fillna(0)



In [41]:
df_策略分布 = pd.read_excel(Public.readFile("持仓日报表模板.xlsx"),sheet_name="策略分布模板")
df_策略分布.iloc[0,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="股票多头"]["持仓市值（万元）"])
df_策略分布.iloc[1,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="指数增强"]["持仓市值（万元）"])
df_策略分布.iloc[2,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="空气指增"]["持仓市值（万元）"])
df_策略分布.iloc[3,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="量化择时"]["持仓市值（万元）"])
df_策略分布.iloc[4,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="量化对冲"]["持仓市值（万元）"])
df_策略分布.iloc[5,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="多策略灵活配置"]["持仓市值（万元）"])
df_策略分布.iloc[6,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="宏观对冲"]["持仓市值（万元）"])
df_策略分布.iloc[7,1] = sum(df_底层资产私募配置情况.loc[df_底层资产私募配置情况["策略类型"]=="量化期货"]["持仓市值（万元）"])

df_策略分布.iloc[0,2] = format(df_策略分布.iloc[0,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")
df_策略分布.iloc[1,2] = format(df_策略分布.iloc[1,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")
df_策略分布.iloc[2,2] = format(df_策略分布.iloc[2,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")
df_策略分布.iloc[3,2] = format(df_策略分布.iloc[3,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")
df_策略分布.iloc[4,2] = format(df_策略分布.iloc[4,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")
df_策略分布.iloc[5,2] = format(df_策略分布.iloc[5,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")
df_策略分布.iloc[6,2] = format(df_策略分布.iloc[6,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")
df_策略分布.iloc[7,2] = format(df_策略分布.iloc[7,1]/df_业务资产及盈亏情况.iloc[-1,1],".2%")

df_策略分布.to_excel("Output_Data/mid_4.xlsx")